In [ ]:
import os
os.chdir('../')

import numpy as np
import tensorflow as tf
from scipy.misc import imread, imresize

from hart.data import disp
from hart.data.kitti.tools import get_data
from hart.model import util
from hart.model.attention_ops import FixedStdAttention
from hart.model.eval_tools import log_norm, log_ratios, log_values, make_expr_logger
from hart.model.tracker import HierarchicalAttentiveRecurrentTracker as HART
from hart.model.nn import AlexNetModel, IsTrainingLayer
from hart.train_tools import TrainSchedule, minimize_clipped

import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
alexnet_dir = 'checkpoints'
img_dir = 'imgs'
checkpoint_path = 'checkpoints/kitti/pretrained/model.ckpt-347346'

img_dir = '../../data/kitti_tracking/small'
label_dir = '../../data/kitti_tracking/label_02'

batch_size = 1
img_size = 187, 621, 3
crop_size = 56, 56, 3

rnn_units = 100
norm = 'batch'
keep_prob = .75

n_timesteps = 60

img_size, crop_size = [np.asarray(i) for i in (img_size, crop_size)]
keys = ['img', 'bbox', 'presence']

In [ ]:
tf.reset_default_graph()
util.set_random_seed(0)

### Placeholders ####################
train_store, train, test_store, test = get_data(img_dir, label_dir, .8, img_size[:2], n_timesteps, n_timesteps,
             batch_size, n_threads=3, in_memory=True, sample_objects=-100,
             truncated_threshold=1., occluded_threshold=1, reverse=True, mirror=True)

x, y, p = [train[k] for k in keys]
test_x, test_y, test_p = [test[k] for k in keys]

is_training = IsTrainingLayer()
builder = AlexNetModel(alexnet_dir, layer='conv3', n_out_feature_maps=5, upsample=False, normlayer=norm,
                       keep_prob=keep_prob, is_training=is_training)

model = HART(x, y[0], p[0], batch_size, crop_size, builder, rnn_units,
             bbox_gain=[-4.78, -1.8, -3., -1.8],
             zoneout_prob=(.05, .05),
             normalize_glimpse=True,
             attention_module=FixedStdAttention,
             debug=True,
             transform_init_features=True,
             transform_init_state=True,
             dfn_readout=True,
             feature_shape=(14, 14),
             is_training=is_training)

tsummediou, tiou_wegihts = model.iou(y, tf.cast(p, tf.bool), True, False, start_t=0)

In [ ]:
print 'Num Train Batches: {}'.format(train_store.n_batches_per_epoch)
print 'Num Test Batches: {}'.format(test_store.n_batches_per_epoch)

In [ ]:
saver = tf.train.Saver()
sess = tf.Session()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [ ]:
sess.run(tf.global_variables_initializer())
saver.restore(sess, checkpoint_path)
model.test_mode(sess)

In [ ]:
def compute_iou(store, fd_dict=None, fraction=1.):
    ws = np.zeros(store.n_timesteps)
    ious = np.zeros(store.n_timesteps)
    n_batches = int(fraction * store.n_batches_per_epoch)
   
    for i in xrange(n_batches):
        print '\r{} / {}'.format(i+1, n_batches),
        fd = None
        if fd_dict:
            fd = sess.run(fd_dict)
        a, b = sess.run([tsummediou, tiou_wegihts], fd)
        ious[:a.shape[0]] += a
        ws[:b.shape[0]] += b
        
    print
    ious /= ws
    return ious

In [ ]:
train_ious = compute_iou(train_store, fraction=.2)
print train_ious

test_fd = {x: test_x, y: test_y, p: test_p}
test_ious = compute_iou(test_store, test_fd, fraction=1.)
print test_ious

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(train_ious, 'r', lw=2, label='train')
ax.plot(test_ious, 'g', lw=2, label='test')
ax.set_ylabel('IoU')
ax.set_xlabel('timestep')
ax.grid(True)
ax.legend(loc=0)
ax.set_title('IoU vs Time')
ax.set_xlim([0, n_timesteps - 1])
fig.tight_layout()